# Create geopackage and lookup table of all geographies of interest to MORPC

## Introduction

This script creates a geopackage that contains feature classes for all geographies of interest to MORPC, and also creates a lookup table that lists all of the geographies and their identification variables.

**As of June 2025, only Census-derived geographies are included.**

This process is dependent on upstream processes. See the "Prerequisites" section below.

The workflow defined herein is identified as workflow ID #91 in the the [Data Team Master Document List](https://morpc1.sharepoint.com/:x:/s/GISteam/EfC4j3HhohZCrSZzxJdyt5cBFEqVD7zHick8ZW0INqgCYA?e=0WhrAI). References to document list identifiers are denoted by a number in brackets, e.g. [91].

## Process outline

  1. Load standardized geographies produced by upstream processes.
  2. Generate a lookup table that includes the combined geographies.
  3. Export lookup table to a file.
  4. Export collected standardized geographies in a single geodatabase.
  5. Create resource files for each of the outputs.

## Prerequisites and usage notes

  - Outputs of one or more upstream workflows must be available at the indicated paths. Make sure that those outputs are up to date prior to running this script. 
  - This script includes several intentional RuntimeError instances that may be triggered to alert the user to conditions that may require their attention. If the script triggers one of these errors, review the error, verify that the condition is acceptable or resolve any issues, then proceed.

## Setup

### Import required packages

In [ ]:
import os
import fiona
import pandas as pd
import geopandas as gpd
import morpc

### User-specified parameters

In [ ]:
# Census geographies are obtained from the outputs of the morpc-censustiger-standardize workflow, which produces
# a new vintage of data when the Census data is released each year.
CENSUS_GEOS_VINTAGE = 2024

# OUTPUT_FILE_SUFFIX is a string that will be appended to the output filenames to differentiate the output
# from other outputs generated by the script. To omit the suffix, set OUTPUT_FILE_SUFFIX = None.
OUTPUT_FILE_SUFFIX = None
# OUTPUT_FILE_SUFFIX = "2020"
# OUTPUT_FILE_SUFFIX = str(CENSUS_GEOS_VINTAGE)   # Use vintage year as suffix

# When DELETE_EXISTING_OUTPUT_DATA == True, the output geopackage will be deleted prior to running the rest of the script.
# This should be the standard behavior to avoid legacy junk building up in the geopackage if, for example, the
# layer names change, however you may want to preserve the existing geodatabase if you are certain all of the existing
# content is valid and you only want to update select layers.
DELETE_EXISTING_OUTPUT_DATA = True

# You can change where the output data is stored by changing the following directory and file names.  This 
# typically is not necessary and may break other scripts that depend on outputs from this one.
OUTPUT_DIR = "./output_data"

### Static parameters

In [ ]:
# Create a list of layers to be omitted from the lookup table.  These will still be present in the GeoPackage.
LOOKUP_TABLE_OMIT_LAYERS = [
    "PLACE","PLACE-COUNTY","COUNTY-TOWNSHIP-REMAINDER",                 # These are superseded by MORPC-specific layers
    "COUNTY-TRACT","COUNTY-TRACT-BG","COUNTY-TRACT-BG-BLOCK","ZCTA5"    # Lookup is not typically required for these
]

# Create a dictionary to collect the spatial data layers
combinedLayers = {}

### Define inputs

The following datasets are required by this notebook.

#### MORPC counties reference data [81]

Reference data for counties in the MORPC region will be loaded automatically as a morpc.countyLookup() object (see below).

#### Standardized Census geographies [228-255]

In [ ]:
CENSUS_GEOS_RESOURCE_PATH = "../morpc-censustiger-standardize/output_data/morpc-standardgeos-census-{}.resource.yaml".format(CENSUS_GEOS_VINTAGE)
print("Resource file: {}".format(CENSUS_GEOS_RESOURCE_PATH))

#### MPO boundary

In [ ]:
MPO_GEO_DATA_PATH = "../morpc-arcsde-fetch/output_data/morpcSDEProduction.gdb"
MPO_GEO_DATA_LAYER = "Bndy_MPO2020boundary"
print("Data: {0}, layer={1}".format(MPO_GEO_DATA_PATH, MPO_GEO_DATA_LAYER))

#### GridMAZ boundaries

In [ ]:
GRIDMAZ_GEO_DATA_PATH = os.path.normpath(r"\\filedfs1\Trans\ArcGIS\CORE\Land Use Model\2024updates\GridTAZMAZ.gdb")
GRIDMAZ_GEO_DATA_LAYER = "GridMAZ2020"
print("Data: {0}, layer={1}".format(GRIDMAZ_GEO_DATA_PATH, GRIDMAZ_GEO_DATA_LAYER))

#### MAZ boundaries

In [ ]:
MAZ_GEO_DATA_PATH = os.path.normpath(r"\\filedfs1\Trans\ArcGIS\CORE\Land Use Model\2024updates\GridTAZMAZ.gdb")
MAZ_GEO_DATA_LAYER = "MAZspecial20"
print("Data: {0}, layer={1}".format(MAZ_GEO_DATA_PATH, MAZ_GEO_DATA_LAYER))

#### TAZ boundaries

In [ ]:
TAZ_GEO_DATA_PATH = os.path.normpath(r"\\filedfs1\Trans\ArcGIS\CORE\Land Use Model\2024updates\GridTAZMAZ.gdb")
TAZ_GEO_DATA_LAYER = "TAZspecial20"
print("Data: {0}, layer={1}".format(TAZ_GEO_DATA_PATH, TAZ_GEO_DATA_LAYER))

### Define outputs

#### Create output data directory

Create output data directory if it doesn't exist.

In [ ]:
outputDir = os.path.normpath(OUTPUT_DIR)
if not os.path.exists(outputDir):
    os.makedirs(outputDir)   

#### Geopackage of collected geographies [376-400]

In [ ]:
GPKG_FILENAME = "morpc-geos.gpkg"
if(OUTPUT_FILE_SUFFIX != None):
    GPKG_FILENAME = GPKG_FILENAME.replace(".gpkg", "-{}.gpkg".format(OUTPUT_FILE_SUFFIX))
GPKG_PATH = os.path.join(outputDir, GPKG_FILENAME)
GPKG_RESOURCE_PATH = GPKG_PATH.replace(".gpkg",".resource.yaml")
print("Data: {}".format(GPKG_PATH))
print("Resource file: {}".format(GPKG_RESOURCE_PATH))

#### Lookup table for collected geographies [375]

In [ ]:
LOOKUP_TABLE_FILENAME = "morpc-geos-lookup.csv"
LOOKUP_TABLE_SCHEMA_PATH = os.path.join(outputDir, LOOKUP_TABLE_FILENAME.replace(".csv",".schema.yaml"))
if(OUTPUT_FILE_SUFFIX != None):
    LOOKUP_TABLE_FILENAME = LOOKUP_TABLE_FILENAME.replace(".csv", "-{}.csv".format(OUTPUT_FILE_SUFFIX))
LOOKUP_TABLE_PATH = os.path.join(outputDir, LOOKUP_TABLE_FILENAME)
LOOKUP_TABLE_RESOURCE_PATH = LOOKUP_TABLE_PATH.replace(".csv",".resource.yaml")
print("Data: {}".format(LOOKUP_TABLE_PATH))
print("Schema: {}".format(LOOKUP_TABLE_SCHEMA_PATH))
print("Resource file: {}".format(LOOKUP_TABLE_RESOURCE_PATH))

## Delete existing output data (maybe)

If DELETE_EXISTING_OUTPUT_DATA == True, delete the existing geopackage for a fresh start.

In [ ]:
if DELETE_EXISTING_OUTPUT_DATA == True:
    if(os.path.exists(GPKG_PATH)):
        try:
            os.unlink(GPKG_PATH)
            print("INFO | Deleted existing geopackage at path {}".format(GPKG_PATH))
        except Exception as e:
            print("ERROR | Failed to delete existing geopackage at path {0}: {1}".format(GPKG_PATH, e))
            raise RuntimeError
    else:
        print("INFO | No existing geopackage found at path {}".format(GPKG_PATH))                                                                    

## Load lookup table schema

In [ ]:
lookupTableSchema = morpc.frictionless.load_schema(LOOKUP_TABLE_SCHEMA_PATH)
lookupTableSchema

## Create empty lookup table

In [ ]:
lookupTable = pd.DataFrame(columns=lookupTableSchema.field_names)
lookupTable = lookupTable.astype({key:value for key, value in zip(lookupTableSchema.field_names, lookupTableSchema.field_types)})
display(lookupTable)
lookupTable.dtypes

## Load and transform the input data

### Load county reference data

### Census geographies

Validate census geographies

In [ ]:
morpc.frictionless.validate_resource(CENSUS_GEOS_RESOURCE_PATH)

Load census geographies resource file.

In [ ]:
censusResource = morpc.frictionless.load_resource(CENSUS_GEOS_RESOURCE_PATH)
censusResource

Obtain the path to the data file from the resource file.

In [ ]:
censusDataPath = os.path.join(os.path.dirname(CENSUS_GEOS_RESOURCE_PATH), censusResource.path)
print("Data: {}".format(censusDataPath))

Get a list of the layers in the geopackage.

In [ ]:
censusLayers = fiona.listlayers(censusDataPath)
censusLayers

Add each layer to the collection of combined layers and append the geographic identifiers to the lookup table.  Preserve the layers as-is, but before appending the records to the lookup table do the following:
  - Extract only the fields that are required for the output and cast them as the appropriate data types.
  - Omit certain Census layers that are superseded by derivative layers maintained by MORPC
  - Omit townships in the JURIS layer because they are superseded by townships in the JURIS-COUNTY layer.
  - Populate the GEOTYPE field with the hierarchy string that corresponds to the sumlevel.
  - Populate the SOURCE field. As of 11/2024, this input contains a mix of census-defined geographies and MORPC-defined geographies.  Ideally the MORPC-defined geographies would be collected in a different input.  For the moment, determine the source by looking for an M prefix in the sumlevel.  Assign those to MORPC and all others to CENSUS.

In [ ]:
sourceMap = {value["hierarchy_string"]: value["authority"].upper() for key, value in zip(morpc.SUMLEVEL_DESCRIPTIONS.keys(), morpc.SUMLEVEL_DESCRIPTIONS.values())}
for layer in censusLayers:
    print("Preparing layer: {}".format(layer))
    # Read the layer from the geopackage
    gdf = gpd.read_file(censusDataPath, layer=layer)
    # Add the layer to the collection as-is
    combinedLayers[layer] = gdf
    # For the lookup table, omit the records in several Census-defined layers which are superseded by 
    # records in MORPC-defined layers with additional attributes.
    if(layer in LOOKUP_TABLE_OMIT_LAYERS):
        continue
    # Extract only the fields that appear in the output schema
    temp = gdf.copy().filter(items=lookupTableSchema.field_names, axis="columns")
    # Populate GEOTYPE field
    temp["GEOTYPE"] = temp["SUMLEVEL"].map(morpc.HIERARCHY_STRING_LOOKUP)
    # Populate SOURCE field
    temp["SOURCE"] = sourceMap[layer]
    # Cast fields as the appropriate type
    temp = morpc.cast_field_types(temp, lookupTableSchema, handleMissingFields="add", verbose=False)
    # Add the prepared records to the lookup table.
    lookupTable = pd.concat([lookupTable, temp], axis="index")

### MPO boundary

Load the data.

In [ ]:
gdf = morpc.load_spatial_data(MPO_GEO_DATA_PATH, layerName=MPO_GEO_DATA_LAYER)

Add missing fields required for the lookup table.

In [ ]:
gdf["SUMLEVEL"] = morpc.SUMLEVEL_LOOKUP["REGIONMPO"]
gdf["GEOID"] = morpc.CONST_REGIONS_GEOID["REGIONMPO"]
gdf["GEOIDFQ"] = gdf["SUMLEVEL"] + "0000US" + gdf["GEOID"]
gdf["GEOTYPE"] = "REGIONMPO"
gdf["SOURCE"] = "MORPC"

Create a dataframe to construct the lookup table.

In [ ]:
df = gdf.filter(items=lookupTableSchema.field_names, axis="columns")

Clean up the fields to be included in the spatial layer.

In [ ]:
gdf = gdf.filter(items=["GEOIDFQ","NAME","GEOID","SUMLEVEL","geometry"], axis="columns")

Add the layer to the collection.

In [ ]:
combinedLayers["REGIONMPO"] = gdf

For the lookup table, cast fields as the appropriate type and add remaining missing fields.

In [ ]:
df = morpc.cast_field_types(df, lookupTableSchema, handleMissingFields="add", verbose=False)

Add the prepared records to the lookup table.

In [ ]:
lookupTable = pd.concat([lookupTable, df], axis="index")

### GridMAZ

Load the data.

In [ ]:
gdf = morpc.load_spatial_data(GRIDMAZ_GEO_DATA_PATH, layerName=GRIDMAZ_GEO_DATA_LAYER)

Add missing fields.

In [ ]:
gdf["SUMLEVEL"] = morpc.SUMLEVEL_LOOKUP["COUNTY-TAZ-MAZ-GRIDMAZ"]
gdf["GEOID"] = gdf["GridMAZ20"].astype("int").astype("str").str.zfill(8)
gdf["GEOIDFQ"] = gdf["SUMLEVEL"] + "0000US" + gdf["GEOID"]
gdf["COUNTYFP"] = gdf["CntyFIPS"]

Clean up the types of existing fields.

In [ ]:
gdf["TAZ2020"] = gdf["TAZ2020"].astype("Int64")
gdf["MAZ2020"] = gdf["MAZ2020"].astype("Int64")
gdf["GRID_ID"] = gdf["GRID_ID"].astype("Int64")
gdf["GridMAZ20"] = gdf["GridMAZ20"].astype("Int64")

Clean up the fields to be included in the spatial layer.

In [ ]:
gdf = gdf.filter(items=["GEOIDFQ","GEOID","SUMLEVEL","COUNTYFP","TAZ2020","MAZ2020","GridMAZ20","GRID_ID","geometry"], axis="columns")

Inspect the data.

In [ ]:
gdf.head()

Add the layer to the collection.

In [ ]:
combinedLayers["COUNTY-TAZ-MAZ-GRIDMAZ"] = gdf

Do not add the GridMAZ to the lookup table.

### MAZ

Load the data.

In [ ]:
gdf = morpc.load_spatial_data(MAZ_GEO_DATA_PATH, layerName=MAZ_GEO_DATA_LAYER)

As of June 2025, some of the records have null values in the MAZ2020 field. These MAZ were located on the periphery of the 10-county region.

In [ ]:
gdf.loc[gdf["MAZ2020"].isna()].explore(color="red")

Add missing fields.  Fill GEOID and GEOIDFQ will null values when MAZ2020 is null.

In [ ]:
gdf["SUMLEVEL"] = morpc.SUMLEVEL_LOOKUP["COUNTY-TAZ-MAZ"]
gdf["GEOID"] = gdf["MAZ2020"].apply(lambda x:(None if pd.isna(x) else str(int(x)).zfill(8)))
gdf["GEOIDFQ"] = gdf["GEOID"].apply(lambda x:(None if pd.isna(x) else (morpc.SUMLEVEL_LOOKUP["COUNTY-TAZ-MAZ"] + "0000US" + x)))
gdf["COUNTYFP"] = gdf["county"].map(morpc.CONST_COUNTY_EXPAND).map(morpc.CONST_COUNTY_NAME_TO_ID).apply(lambda x:x[2:5]).astype("str")

Clean up the types of existing fields.

In [ ]:
gdf["TAZ2020"] = gdf["TAZ2020"].astype("Int64")
gdf["MAZ2020"] = gdf["MAZ2020"].astype("Int64")

Clean up the fields to be included in the spatial layer.

In [ ]:
gdf = gdf.filter(items=["GEOIDFQ","GEOID","SUMLEVEL","COUNTYFP","TAZ2020","MAZ2020","geometry"], axis="columns")

Inspect the data.

In [ ]:
gdf.head()

Add the layer to the collection.

In [ ]:
combinedLayers["COUNTY-TAZ-MAZ"] = gdf

Do not add the MAZ to the lookup table.

### TAZ

Load the data.

In [ ]:
gdf = morpc.load_spatial_data(TAZ_GEO_DATA_PATH, layerName=TAZ_GEO_DATA_LAYER)

Add missing fields.

In [ ]:
gdf["SUMLEVEL"] = morpc.SUMLEVEL_LOOKUP["COUNTY-TAZ"]
gdf["GEOID"] = gdf["TAZ2020"].astype("int").astype("str").str.zfill(8)
gdf["GEOIDFQ"] = gdf["SUMLEVEL"] + "0000US" + gdf["GEOID"]
gdf["COUNTYFP"] = gdf["county"].map(morpc.CONST_COUNTY_EXPAND).map(morpc.CONST_COUNTY_NAME_TO_ID).apply(lambda x:x[2:5]).astype("str")

Clean up the types of existing fields.

In [ ]:
gdf["TAZ2020"] = gdf["TAZ2020"].astype("int")

Clean up the fields to be included in the spatial layer.

In [ ]:
gdf = gdf.filter(items=["GEOIDFQ","GEOID","SUMLEVEL","COUNTYFP","TAZ2020","geometry"], axis="columns")

Inspect the data.

In [ ]:
gdf.head()

Add the layer to the collection.

In [ ]:
combinedLayers["COUNTY-TAZ"] = gdf

Do not add the TAZ to the lookup table.

## Prepare lookup table for export

Ensure that the lookup table contains only the columns defined in the schema.

In [ ]:
lookupTable = lookupTable.filter(items=lookupTableSchema.field_names, axis="columns")

Ensure that all fields are cast as the type specified in the schema.

In [ ]:
lookupTable = morpc.cast_field_types(lookupTable, lookupTableSchema)

Sort the data by GEOIDFQ.  This effectively sorts the data by SUMLEVEL then by GEOID.

In [ ]:
lookupTable = lookupTable.sort_values("GEOIDFQ")

Use the fully-qualified GEOID as the index.

In [ ]:
lookupTable = lookupTable.set_index(lookupTableSchema.primary_key)

Verify that there is only one record associated with each GEODID.

In [ ]:
lookupTable.index.is_unique

Preview the data. It is expected that there are null values in some fields.

In [ ]:
lookupTable.head()

In [ ]:
lookupTable.tail(10)

## Export data

### Export the collected layers as a single geopackage

Write each of the standardized layers to the output geopackage.

In [ ]:
combinedLayers["REGIONMPO"]

In [ ]:
for layer in combinedLayers.keys():
    print("INFO: Writing layer {0} to output file {1}".format(layer, GPKG_PATH))
    temp = combinedLayers[layer].copy()
    temp.to_file(GPKG_PATH, layer=layer)

Create a Frictionless Resource file to describe the data.

In [ ]:
description = "This geopackage contains a set of geographies for the MORPC 15-county region derived from several sources. \
The geographies have been standardized for use by MORPC scripts, for example by reprojecting to a standard coordinate reference \
system and adding certain MORPC-specific fields. For more information, see the morpc-geos-collect workflow and the upstream \
workflows referenced therein. The included geographies are as follows: {}".format(", ".join(combinedLayers.keys())) 

morpc.frictionless.create_resource(
    os.path.basename(GPKG_PATH), 
    resourcePath=GPKG_RESOURCE_PATH,
    title="Complete collection of geographies of interest to MORPC", 
    description=description,
    computeHash=True,
    computeBytes=True,
    ignoreSchema=True,
    writeResource=True,
    validate=True
)

### Export the lookup table

Write the lookup table to a CSV file.

In [ ]:
lookupTable.to_csv(LOOKUP_TABLE_PATH, index=True)

Create a Frictionless Resource file to describe the data.

In [ ]:
description = "This table provides the identifiers for a set of geographies for the MORPC 15-county region derived from several \
sources. The geographies themselves are available in an associated geopackage. For more information, see the \
morpc-geos-collect workflow.  Identifiers are included for the following geographies: {}".format(", ".join(combinedLayers.keys())) 

morpc.frictionless.create_resource(
    os.path.basename(LOOKUP_TABLE_PATH), 
    resourcePath=LOOKUP_TABLE_RESOURCE_PATH,
    schemaPath=os.path.basename(LOOKUP_TABLE_SCHEMA_PATH),
    title="Identifiers for geographies of interest to MORPC", 
    description=description,
    computeHash=True,
    computeBytes=True,
    writeResource=True,
    validate=True
)